In [ ]:
import torch
import matplotlib.pyplot as plt
from utils.defects import S_cal
import numpy as np

from kinetic_solver import KineticSolver, KineticData
from nematic_env import ActiveNematicEnv

In [ ]:
t = 50
print('t = ', t)

In [ ]:
action = (-1, 0, 1, 1, 1, 1)
light_mat = ActiveNematicEnv._action2light(10, action, grid_size=256)
import matplotlib.pyplot as plt
# plt.imshow(light_mat, cmap='gray', alpha=0.5)
# # set the colorbar from 0 to 10
# plt.clim(0, 10)
# plt.colorbar()
# plt.show()
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
# ax.imshow(light_mat, alpha=0.5)
# set the colorbar from 0 to 10
fig.colorbar(ax.imshow(light_mat, cmap='gray', alpha=1, vmin=0, vmax=10))
# title = 't = ' + str(t)
ax.set_title(str(action))
num_test = 20
# show num_test below x axis
ax.text(-5, -5, str(num_test), ha='center', va='center')
# set x label
ax.set_xlabel('sss'+str(num_test))
# cb = fig.colorbar(ax.imshow(light_mat, cmap='gray', alpha=0.5))
# # set the colorbar from 0 to 10
# cb.set_clim(0, 10)

plt.show()


In [ ]:
print(0%8)